In [39]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Bring biomass data into dataframe

In [40]:
Hudson = pd.read_excel('D:\APSIMX2\Prototypes\Lucerne\Hudson.xlsx',sheetname='Hudson')
Observed = pd.read_excel('D:\APSIMX2\Prototypes\Lucerne\observed.xlsx',sheetname='Observed')
Nekia = pd.read_excel('D:\APSIMX2\Prototypes\Lucerne\Nekia.xlsx',sheetname='Nekia')
LucerneAllObs= pd.read_excel('D:\APSIMX2\Prototypes\Lucerne\LucerneAllObs.xlsx',sheetname='LucerneAllObs')




In [41]:
AllDataTabs = [Hudson,Observed,Nekia,LucerneAllObs]

In [ ]:
for Tab in AllDataTabs:
    print Tab.ix[:,'Clock.Today'].dty

## Make a column index

In [42]:
ColumnIndex = Hudson.columns
for Table in AllDataTabs:
    ColumnIndex = ColumnIndex  | Table.columns  # the union of the merged data frame and the next in the list

## Make row index

In [43]:
RowIndexData = Hudson.ix[:,Hudson.columns.isin(['SimulationName','Clock.Today'])]
for Table in AllDataTabs:
    RowIndexData = pd.concat([RowIndexData,Table.ix[:,Table.columns.isin(['SimulationName','Clock.Today'])]],axis=0)
RowIndex = RowIndexData.drop_duplicates()
RowIndex.set_index(['SimulationName','Clock.Today'],inplace=True)

C:\Users\yxm20\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.
C:\Users\yxm20\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


## Make empty Data frame

In [44]:
ObsDataFrame = pd.DataFrame(index=RowIndex.index, columns=ColumnIndex)

## Set indexes in the smaller tables and deal to duplicate rows

In [45]:
from collections import Counter
def MakeMean(ser):
    Series = pd.Series(index=ser.columns)
    for Group in ser.columns:
        if isinstance(Group, float):
            Series[Group] = ser[Group].mean()
        else:
            Series[Group] = Counter(ser[Group]).most_common(1)[0][0]
    return Series        

In [46]:
CompressedData = []
for Table in AllDataTabs:
    Table.set_index(['SimulationName','Clock.Today'],inplace=True)
    Compressed = Table.groupby(level=['SimulationName','Clock.Today']).apply(MakeMean)
    CompressedData.append(Compressed)

In [47]:
for DataTable in range(len(CompressedData)):
    ObsDataFrame.update(CompressedData[DataTable])


In [49]:
ObsDataFrame.to_excel('D:\APSIMX2\Prototypes\Lucerne\ObsAll.xlsx',merge_cells=False,sheet_name='ObsAll')